In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pymongo import MongoClient
#import sqlite3

In [2]:
site = 'https://quotes.toscrape.com'

def getQuotesContent(site):
    response = requests.get(site) 
    return response.content
    

In [3]:
def getQuotes(content):
    res = []
    soup = bs(content, 'html.parser')            
    quotes = soup.find_all('div', attrs={'class': 'quote'})
    for quote in quotes:
        res.append({"text": quote.find('span', attrs={'class': 'text'}).text, 
                   "author": quote.find('small', attrs={'class': 'author'}).text})       
    return res

def getQuotesDepth(site, depth):
    res = []
    for i in range(1, depth+1):
        res = res + getQuotes(getQuotesContent(f"{site}/page/{i}/"))
    return res

qts = getQuotesDepth(site,3)

### Сохранить в БД

In [4]:
client = MongoClient('mongodb://localhost:27017/')

In [5]:
client.list_database_names()

['admin', 'config', 'local', 'news']

In [6]:
db = client.quotes
#db.quotes.drop()

In [7]:
# db.quotes.insert_many(qts)
print(f"записей в БД: {db.quotes.count_documents({})}")
for q in qts:
    if db.quotes.count_documents({"text": q["text"]}) == 0:
        db.quotes.insert_one(q)
print(f"записей в БД: {db.quotes.count_documents({})}")

записей в БД: 0
записей в БД: 30


In [8]:
client.list_database_names()

['admin', 'config', 'local', 'news', 'quotes']

In [9]:
#db.quotes.drop()

In [10]:
r = db.quotes.find({"author": 'Albert Einstein'})
for a in r:
    print(a["text"],a["author"])

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein
“Try not to become a man of success. Rather become a man of value.” Albert Einstein
“If you can't explain it to a six year old, you don't understand it yourself.” Albert Einstein
“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.” Albert Einstein
“Logic will get you from A to Z; imagination will get you everywhere.” Albert Einstein
